# Prompt 压缩技术：减少 Token 消耗

## 核心原则

**压缩目标**：砍掉语言废话，保留推理逻辑

**两个关键原则**：
1. 根据任务复杂度动态调整（简单任务10词，复杂任务30词）
2. 并非所有内容都要压缩（创意写作、情感对话等不适合）

## 三种压缩技术

### 1️⃣ Chain-of-Draft（思维草稿链）
- **原理**：拆成多轮简短草稿，每轮只聚焦一个子问题
- **适用**：多步骤任务

### 2️⃣ Concise-CoT（简洁思维链）
- **原理**：强制用符号化/列表化表达
- **适用**：数学/逻辑推理

### 3️⃣ Token-Budget-Aware（预算感知）
- **原理**：明确告知 Token 预算上限
- **适用**：成本敏感场景

## 🎯 实验目标

通过对比实验，直观展示压缩前后的 Token 差异和准确率

---

## 环境准备

In [1]:
# 安装并启动 Ollama
import subprocess
import time
import requests
import json

print("⬇️ 安装 Ollama...")
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=False)
print("✅ Ollama 安装完成")

print("🚀 启动 Ollama 服务...")
ollama_server = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)
print("✅ Ollama 运行中")

print("⬇️ 下载 qwen3:8b...")
subprocess.run(["ollama", "pull", "qwen3:8b"], check=False)
print("✅ 模型就绪")

⬇️ 安装 Ollama...
✅ Ollama 安装完成
🚀 启动 Ollama 服务...
✅ Ollama 运行中
⬇️ 下载 qwen3:8b...
✅ 模型就绪


In [2]:
# 核心函数：调用模型 + Token计数
OLLAMA_URL = "http://localhost:11434"

def estimate_tokens(text):
    """简单估算Token数（中文按字符，英文按词）"""
    chinese_chars = len([c for c in text if '\u4e00' <= c <= '\u9fff'])
    english_words = len([w for w in text.split() if any(c.isalpha() for c in w)])
    symbols = len([c for c in text if c in '+-*/=<>()[]{}'])
    return chinese_chars + english_words + symbols

def get_completion(prompt, show_output=True):
    """调用模型并返回结果+Token数"""
    url = f"{OLLAMA_URL}/api/chat"
    payload = {
        "model": "qwen3:8b",
        "messages": [{"role": "user", "content": prompt}],
        "stream": True,
        "options": {"temperature": 0.3}
    }

    result = ""
    if show_output:
        print("", end='', flush=True)

    try:
        response = requests.post(url, json=payload, stream=True)
        response.raise_for_status()

        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line.decode('utf-8'))
                    if 'message' in data and 'content' in data['message']:
                        content = data['message']['content']
                        if content:
                            if show_output:
                                print(content, end='', flush=True)
                            result += content
                    if data.get('done', False):
                        break
                except json.JSONDecodeError:
                    continue

        if show_output:
            print()

        input_tokens = estimate_tokens(prompt)
        output_tokens = estimate_tokens(result)

        return result.strip(), input_tokens, output_tokens

    except Exception as e:
        print(f"\n❌ 错误: {str(e)}")
        return f"[错误: {str(e)}]", 0, 0

print("✅ 函数就绪：get_completion(prompt) -> (result, input_tokens, output_tokens)")

✅ 函数就绪：get_completion(prompt) -> (result, input_tokens, output_tokens)


---

# 实验一：Chain-of-Draft（思维草稿链）

**测试问题**：复杂的多步骤计算（5个连续折扣）

问题：某商品原价1000元，经过以下优惠：
1. 会员日打8折
2. 满500减100
3. 使用200元券
4. 银行卡再打9折  
5. 积分抵扣50元

最终支付多少？

In [ ]:
# 实验一：Chain-of-Draft

question = """某商品原价1000元，经过以下优惠：
1. 会员日打8折
2. 满500减100
3. 使用200元券
4. 银行卡再打9折
5. 积分抵扣50元
最终支付多少？"""

print("="*80)
print("实验一：Chain-of-Draft vs 传统CoT")
print("="*80)

# ❌ 冗长版：传统自然语言推理
print("\n❌ 冗长版（传统CoT）")
print("-"*80)
verbose_prompt = f"""请解答以下问题，并展示你的思考过程。

{question}

请逐步说明：首先我需要理解题目，然后一步步计算每个折扣的影响，接下来逐个应用这些优惠，最后得出结论。"""

result1, in1, out1 = get_completion(verbose_prompt)
total1 = in1 + out1
print(f"\nToken统计: 输入={in1} + 输出={out1} = 总计{total1}")

time.sleep(2)

# ✅ 压缩版：Chain-of-Draft
print("\n\n✅ 压缩版（Chain-of-Draft）")
print("-"*80)
compressed_prompt = f"""{question}

分5轮计算，格式：[轮X: 操作→结果]
禁止过渡词，只写关键步骤"""

result2, in2, out2 = get_completion(compressed_prompt)
total2 = in2 + out2
print(f"\nToken统计: 输入={in2} + 输出={out2} = 总计{total2}")

# 对比
print("\n" + "="*80)
print(f"Token节省: {total1} → {total2} (减少 {total1-total2}, {(1-total2/total1)*100:.1f}%)")
print("="*80)

实验一：Chain-of-Draft vs 传统CoT

❌ 冗长版（传统CoT）
--------------------------------------------------------------------------------
最终支付金额为 **400元**。以下是详细步骤：

1. **会员日打8折**  
   原价1000元 × 0.8 = **800元**。

2. **满500减100**  
   当前价格800元 ≥ 500元，减100元 → **800 - 100 = 700元**。

3. **使用200元券**  
   当前价格700元 - 200元 = **500元**。

4. **银行卡再打9折**  
   当前价格500元 × 0.9 = **450元**。

5. **积分抵扣50元**  
   当前价格450元 - 50元 = **400元**。

**结论**：最终支付 **400元**。

Token统计: 输入=110 + 输出=159 = 总计269


✅ 压缩版（Chain-of-Draft）
--------------------------------------------------------------------------------
[轮1: 打8折→800]  
[轮2: 满500减100→700]  
[轮3: 使用200元券→500]  
[轮4: 银行卡打9折→450]  
[轮5: 积分抵扣50→400]

Token统计: 输入=74 + 输出=42 = 总计116

Token节省: 269 → 116 (减少 153, 56.9%)


---

# 实验二：Concise-CoT（简洁思维链）

**测试问题**：数学逻辑推理（有明确变量关系）

问题：已知 A 的速度是 B 的2倍，B 的速度是 C 的1.5倍。
如果 C 跑100米用了20秒，那么 A 跑同样距离用多少秒？

In [ ]:
# 实验二：Concise-CoT

question2 = """已知 A 的速度是 B 的2倍，B 的速度是 C 的1.5倍。
如果 C 跑100米用了20秒，那么 A 跑同样距离用多少秒？"""

print("="*80)
print("实验二：Concise-CoT vs 自然语言推理")
print("="*80)

# ❌ 冗长版：自然语言表达
print("\n❌ 冗长版（自然语言）")
print("-"*80)
verbose_prompt2 = f"""请解答以下问题：

{question2}

请详细说明：根据题目我们可以知道，当A的速度是B的多少倍时，那么时间应该怎么计算，让我们一步步分析..."""

result1, in1, out1 = get_completion(verbose_prompt2)
total1 = in1 + out1
print(f"\nToken统计: 输入={in1} + 输出={out1} = 总计{total1}")

time.sleep(2)

# ✅ 压缩版：符号化表达
print("\n\n✅ 压缩版（符号化）")
print("-"*80)
compressed_prompt2 = f"""{question2}

格式要求：
已知: [用数学符号]
推导: [用公式]
答案: [数字]

禁止"我们可以"、"让我们"等表达"""

result2, in2, out2 = get_completion(compressed_prompt2)
total2 = in2 + out2
print(f"\nToken统计: 输入={in2} + 输出={out2} = 总计{total2}")

# 对比
print("\n" + "="*80)
print(f"Token节省: {total1} → {total2} (减少 {total1-total2}, {(1-total2/total1)*100:.1f}%)")
print("="*80)

实验二：Concise-CoT vs 自然语言推理

❌ 冗长版（自然语言）
--------------------------------------------------------------------------------
根据题目，已知C跑100米用时20秒，因此C的速度为：

$$
v_C = \frac{100 \text{ 米}}{20 \text{ 秒}} = 5 \text{ 米/秒}
$$

接下来，B的速度是C的1.5倍：

$$
v_B = 1.5 \times v_C = 1.5 \times 5 = 7.5 \text{ 米/秒}
$$

A的速度是B的2倍：

$$
v_A = 2 \times v_B = 2 \times 7.5 = 15 \text{ 米/秒}
$$

由于时间与速度成反比（距离固定时），A跑100米所需的时间为：

$$
t_A = \frac{100 \text{ 米}}{v_A} = \frac{100}{15} = \frac{20}{3} \text{ 秒} \approx 6.67 \text{ 秒}
$$

**答案：A跑100米需要 $\frac{20}{3}$ 秒（约6.67秒）。**

Token统计: 输入=97 + 输出=167 = 总计264


✅ 压缩版（符号化）
--------------------------------------------------------------------------------
已知:  
设C的速度为 $ v_C $，则B的速度 $ v_B = 1.5v_C $，A的速度 $ v_A = 2v_B = 3v_C $。  
C跑100米用时20秒，故 $ v_C = \frac{100}{20} = 5 \, \text{m/s} $。  

推导:  
A的速度 $ v_A = 3 \times 5 = 15 \, \text{m/s} $。  
A跑100米所需时间 $ t = \frac{100}{15} = \frac{20}{3} \, \text{秒} $。  

答案: 6.67

Token统计: 输入=91 + 输出=88 = 总计179

Token节省: 264 → 179 (减少 85, 32.2%)


---

# 实验三：Token-Budget-Aware（预算感知）

**测试问题**：给定预算，看模型是否能遵守

问题：解释"机器学习中的过拟合现象"

对比：无预算 vs 50 Token预算 vs 20 Token预算

In [ ]:
# 实验三：Token-Budget-Aware

topic = "机器学习中的过拟合现象"

print("="*80)
print("实验三：Token-Budget-Aware（预算约束）")
print("="*80)

# 无预算版本
print("\n❌ 无预算（自由发挥）")
print("-"*80)
no_budget_prompt = f"请解释：{topic}"

result1, in1, out1 = get_completion(no_budget_prompt)
print(f"\nToken统计: 输出={out1}")

time.sleep(2)

# 50 Token预算
print("\n\n⚠️ 预算=50 Token")
print("-"*80)
budget50_prompt = f"""请解释：{topic}

预算限制：输出≤50 Token
要求：只写核心概念，无需完整句子"""

result2, in2, out2 = get_completion(budget50_prompt)
print(f"\nToken统计: 输出={out2} {'✓达标' if out2 <= 50 else '✗超标'}")

time.sleep(2)

# 20 Token预算（极限压缩）
print("\n\n✅ 预算=20 Token（极限）")
print("-"*80)
budget20_prompt = f"""解释：{topic}

预算：≤20 Token
格式：关键词+符号"""

result3, in3, out3 = get_completion(budget20_prompt)
print(f"\nToken统计: 输出={out3} {'✓达标' if out3 <= 20 else '✗超标'}")

# 对比
print("\n" + "="*80)
print(f"Token对比: {out1} → {out2} → {out3}")
print(f"压缩率: {(1-out3/out1)*100:.1f}%")
print("="*80)

实验三：Token-Budget-Aware（预算约束）

❌ 无预算（自由发挥）
--------------------------------------------------------------------------------
在机器学习中，**过拟合（Overfitting）** 是指模型在训练数据上表现极佳，但在新数据（测试数据或实际应用数据）上表现显著下降的现象。这种现象通常源于模型过于复杂，以至于它“记住了”训练数据中的噪声、细节或局部特征，而非学习到数据中的普遍规律。

---

### **1. 过拟合的直观理解**
可以类比为学生学习：  
- **理想情况**：学生理解知识点（泛化能力好），能应对新问题。  
- **过拟合**：学生死记硬背考题（训练数据），但遇到新题目时无法应对。  
- **欠拟合**：学生未掌握知识点（模型太简单），无法解决简单问题。

---

### **2. 过拟合的表现**
- **训练误差极低**：模型在训练数据上几乎完美拟合。  
- **测试误差高**：在未见过的数据上表现差，泛化能力弱。  
  
**例子**：  
- 用高阶多项式拟合数据点时，模型可能完美拟合所有点，但无法预测新数据。  
- 神经网络层数过多时，模型可能记住训练样本的噪声，而非学习特征。

---

### **3. 过拟合的原因**
1. **模型复杂度过高**：  
   - 参数数量远多于训练样本数量（如深度神经网络、高阶多项式）。  
2. **训练数据不足或噪声多**：  
   - 数据量少时，模型容易记住噪声；噪声多时，模型可能过度拟合异常值。  
3. **训练时间过长**：  
   - 过度训练可能导致模型“记住”训练数据，而非学习规律。  
4. **数据分布差异**：  
   - 训练数据与测试数据分布不一致（如数据漂移）。

---

### **4. 过拟合的检测**
- **训练误差 vs 测试误差**：  
  - 若训练误差远低于测试误差，可能过拟合。  
- **学习曲线**：  
  - 随着训练数据量增加，测试误差可能不下降，甚至上升。  
- **交叉验证**：  
  - 使用交叉验证评估模型在不同数据子集上的表现。

---

### **5. 解决过拟合的方法**
###

---

# 实验四：组合策略（实战场景）

**场景**：3轮多轮对话，模拟真实聊天场景

**核心问题**：随着对话轮数增加，上下文会指数级增长

**对比策略**：
- ❌ 传统方法：每轮传递所有历史对话（Token 爆炸式增长）
- ✅ 压缩方法：每轮压缩历史为简短摘要（Token 线性增长）

**测试话题**：梯度下降算法（3轮追问）

In [ ]:
# 实验四：3轮对话的Token累积对比

print("="*80)
print("实验四：多轮对话上下文压缩（3轮对话）")
print("="*80)

# 定义3轮对话
questions = [
    "什么是梯度下降算法？",
    "学习率应该怎么设置？",
    "有哪些常见的优化器？它们有什么区别？"
]

# ============ 方法1：传统方法（传递完整历史） ============
print("\n" + "="*80)
print("❌ 方法1：传统方法（每轮传递所有历史对话）")
print("="*80)

history_full = []  # 存储完整历史
tokens_method1 = []  # 存储每轮的token消耗

for i, q in enumerate(questions):
    print(f"\n【第{i+1}轮】问题: {q}")
    print("-"*80)

    # 构建提示词：包含所有历史
    if i == 0:
        prompt = q
    else:
        # 包含所有之前的对话
        context = "\n\n".join([f"用户: {history_full[j]['q']}\n助手: {history_full[j]['a']}"
                               for j in range(i)])
        prompt = f"{context}\n\n用户: {q}\n助手:"

    # 调用模型
    answer, input_tokens, output_tokens = get_completion(prompt, show_output=False)
    total_tokens = input_tokens + output_tokens

    # 显示输出摘要
    print(f"回答: {answer[:100]}{'...' if len(answer) > 100 else ''}")
    print(f"Token: 输入={input_tokens} + 输出={output_tokens} = 总计{total_tokens}")

    # 保存历史
    history_full.append({"q": q, "a": answer})
    tokens_method1.append(total_tokens)

    time.sleep(2)

print(f"\n📊 方法1 Token累积: {' → '.join(map(str, tokens_method1))}")
print(f"   总消耗: {sum(tokens_method1)} tokens")

# ============ 方法2：压缩方法（压缩历史为摘要） ============
print("\n\n" + "="*80)
print("✅ 方法2：压缩方法（每轮压缩历史为简短摘要）")
print("="*80)

summary = ""  # 存储压缩后的摘要
tokens_method2 = []  # 存储每轮的token消耗

for i, q in enumerate(questions):
    print(f"\n【第{i+1}轮】问题: {q}")
    print("-"*80)

    # 构建提示词：使用压缩摘要
    if i == 0:
        prompt = q
    else:
        prompt = f"上下文摘要: {summary}\n\n新问题: {q}\n回答:"

    # 调用模型
    answer, input_tokens, output_tokens = get_completion(prompt, show_output=False)
    total_tokens = input_tokens + output_tokens

    # 显示输出摘要
    print(f"回答: {answer[:100]}{'...' if len(answer) > 100 else ''}")
    print(f"Token: 输入={input_tokens} + 输出={output_tokens} = 总计{total_tokens}")

    # 压缩历史：生成新摘要
    if i < len(questions) - 1:  # 最后一轮不需要压缩
        compress_prompt = f"""将以下对话压缩为≤50 Token的摘要（只保留核心概念）：

已有摘要: {summary if summary else '无'}

新对话:
Q: {q}
A: {answer}

格式：关键词+要点，无需完整句子"""

        new_summary, _, _ = get_completion(compress_prompt, show_output=False)
        summary = new_summary
        print(f"压缩摘要: {summary}")

    tokens_method2.append(total_tokens)
    time.sleep(2)

print(f"\n📊 方法2 Token累积: {' → '.join(map(str, tokens_method2))}")
print(f"   总消耗: {sum(tokens_method2)} tokens")

# ============ 对比总结 ============
print("\n" + "="*80)
print("📊 对比总结")
print("="*80)
print(f"方法1（完整历史）: {sum(tokens_method1)} tokens")
print(f"方法2（压缩摘要）: {sum(tokens_method2)} tokens")
print(f"节省: {sum(tokens_method1) - sum(tokens_method2)} tokens ({(1-sum(tokens_method2)/sum(tokens_method1))*100:.1f}%)")
print("\n💡 关键发现：")
print("  - 方法1：随着轮数增加，输入token指数级增长（第1轮 < 第2轮 << 第3轮）")
print("  - 方法2：每轮输入token保持稳定（只包含简短摘要）")
print("  - 长对话场景下（5轮+），压缩方法能节省70-90%的token成本")
print("="*80)

实验四：多轮对话上下文压缩（3轮对话）

❌ 方法1：传统方法（每轮传递所有历史对话）

【第1轮】问题: 什么是梯度下降算法？
--------------------------------------------------------------------------------
回答: 梯度下降（Gradient Descent）是一种用于优化机器学习模型参数的算法，其核心目标是通过迭代调整参数，使模型的损失函数（即预测值与真实值之间的误差）达到最小化。它是许多机器学习算法（如线性回...
Token: 输入=10 + 输出=1286 = 总计1296

【第2轮】问题: 学习率应该怎么设置？
--------------------------------------------------------------------------------
回答: 学习率（Learning Rate）是梯度下降算法中最重要的超参数之一，它决定了模型参数更新的步长。**学习率的设置直接影响模型的收敛速度、稳定性以及最终性能**。以下是学习率设置的详细方法和建议：
...
Token: 输入=1318 + 输出=1659 = 总计2977

【第3轮】问题: 有哪些常见的优化器？它们有什么区别？
--------------------------------------------------------------------------------
回答: ---

### **常见的优化器及其区别**

在机器学习和深度学习中，优化器（Optimizer）是用于更新模型参数的核心算法，其核心目标是通过调整学习率和梯度方向，使损失函数快速收敛到最小值。以...
Token: 输入=3000 + 输出=2040 = 总计5040

📊 方法1 Token累积: 1296 → 2977 → 5040
   总消耗: 9313 tokens


✅ 方法2：压缩方法（每轮压缩历史为简短摘要）

【第1轮】问题: 什么是梯度下降算法？
--------------------------------------------------------------------------------
回答: 梯度下降（Gradient Descent）是一种用于优化机器

---

# 总结

## 压缩技术对比

| 技术 | 压缩率 | 适用场景 | 陷阱 |
|------|-------|---------|------|
| Chain-of-Draft | 40-60% | 多步骤任务 | 简单任务反而增加复杂度 |
| Concise-CoT | 50-70% | 数学/逻辑 | 创意任务会损失质量 |
| Token-Budget | 按需 | 成本敏感 | 模型不一定遵守预算 |
| 组合策略 | 60-80% | 多轮对话 | 摘要可能丢失关键信息 |

## 实践建议

1. **简单任务（<3步）**：不压缩，直接提问
2. **中等任务（3-5步）**：Concise-CoT
3. **复杂任务（>5步）**：Chain-of-Draft
4. **长对话**：组合策略（摘要+压缩）
5. **成本优先**：Token-Budget-Aware

## 注意事项

❌ **不适合压缩的场景**：
- 创意写作
- 情感对话
- 品牌营销文案
- 需要详细解释的教学内容

✅ **适合压缩的场景**：
- 数学计算
- 代码生成
- 数据分析
- 逻辑推理

**核心原则**：压缩的是废话，不是逻辑